In [111]:
import pandas as pd
import numpy as np
import re

import os
import gensim
from gensim import models
from gensim.models import Word2Vec, KeyedVectors


from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize,word_tokenize
# Scalers
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

# Models
from sklearn.linear_model import LogisticRegression #logistic regression
from sklearn.linear_model import Perceptron
from sklearn import svm #support vector Machine
from sklearn.ensemble import RandomForestClassifier #Random Forest
from sklearn.neighbors import KNeighborsClassifier #KNN
from sklearn.naive_bayes import GaussianNB #Naive bayes
from sklearn.tree import DecisionTreeClassifier #Decision Tree
from sklearn.model_selection import train_test_split #training and testing data split
from sklearn import metrics #accuracy measure
from sklearn.metrics import confusion_matrix #for confusion matrix
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

# Cross-validation
from sklearn.model_selection import KFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.model_selection import cross_val_predict #prediction
from sklearn.model_selection import cross_validate

# GridSearchCV
from sklearn.model_selection import GridSearchCV

#Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

In [112]:
data=pd.read_csv("Copy of Project.csv")

In [113]:
data.head()

,Timestamp,Select a Role,"1. You have just arrived to the office as an administrative worker at a large accounting firm. It is Monday, 8:30 am, and you have multiple tasks that need to get done before the day is done. Most of the staff in your office are very busy doing other things and will be unable to assist you. Your workday will be over at 2:00 PM.Which of the following tasks would you accomplish first?",2. You are a department manager and you have recently thought of a new procedure that you believe would improve the work process. Some of the employees in your department agree with the change and some do not.One of your employees openly criticizes the idea to your director.,"3. At a marketing meeting with your supervisor and the senior marketing manager you find yourself in the midst of a conflict between them. You know that the two do not get along professionally and that they are inconstant disagreement.They are now arguing about strategies for a new campaign, and are asking you to pick a side.","4. ""You have been working as a salesperson for the past year and have consistently achieved great sales numbers. Due to personal reasons, you have recently been unfocused at work and as a result your work performance has declined. Additionally, due to changes in the market, the sales figures of your team have decreased as well.Your director does not seem to be taking the changes in the market into account and is blaming your poor performance on poor leadership on your part. ""","5. ""You've been working in the same place for the past three years and have managed to work your way up. Lately, you have been feeling that you have reached your potential in the company so you start pursuing options for advancing your career in other companies. You are now in the midst of negotiations for a new position. Rumors that you are leaving have spread in your current work place. ""","6. The company you work for is experiencing financial difficulties. You have thought of a creative solution that will enable it to recruit more clients.However, the downside is that the company will have to let go a stable,loyal but not so profitable client (due to a conflict of interest).Two out of three marketing people agree with you while your manager does not because she believes this is too risky.","7. ""A co-worker is undermining you. Currently, he is at a less senior position than you in the company and hasn't been working there for long. However,he is better educated than you and is also considered a fast learner. You know from a third party that he is interested in taking over some of your responsibilities. ""","8. ""You are a department manager. Two members of your staff are long overdue to participate in a professional training course. The training department manager has informed you that she has chosen employees from a different department instead.Relations with the training department are already problematic. """,...,"16. ""In the past month one of your employees has shown a major decline in sales performance. Although the decline has been ongoing for a couple of months, it has been particularly steep during the past two weeks. In addition, she has started coming in late and seems very frustrated with her work. Her behavior is influencing the atmosphere in the office as she is a popular employee and has been working for the company for the past two years. ""","17. ""You are assigned to work on a project along with another co-worker. He has been working in the department longer than you. Since he has no great ambitions or aspirations to develop professionally, he does not put much effort into your mutual project. ""","18. ""You work in a large audit firm as a consultant. There are five additional members in the team, all working under the supervision of a team manager and the department manager above him.Matt, a co-worker in your staff with whom you share an office, needs your advice regarding a report he is about to present at this week's team mee

In [114]:
del data['Timestamp']


In [115]:
data['1. You have just arrived to the office as an administrative worker at a large accounting firm. It is Monday, 8:30 am, and you have multiple tasks that need to get done before the day is done. Most of the staff in your office are very busy doing other things and will be unable to assist you. Your workday will be over at 2:00 PM.Which of the following tasks would you accomplish first? '].unique()

array(['You receive an Email indicating an impromptu administrative staff meeting starting in 15 minutes.',
       'There are 17 purchase orders to review for completeness. These orders need to be finished by the end of the week.',
       'You receive a note indicating a personal talk with an HR specialist to discuss your work environment between 10:15AM-12:30PM.'],
      dtype=object)

In [116]:
data['1. You have just arrived to the office as an administrative worker at a large accounting firm. It is Monday, 8:30 am, and you have multiple tasks that need to get done before the day is done. Most of the staff in your office are very busy doing other things and will be unable to assist you. Your workday will be over at 2:00 PM.Which of the following tasks would you accomplish first? '].value_counts()

You receive an Email indicating an impromptu administrative staff meeting starting in 15 minutes.                                392
There are 17 purchase orders to review for completeness. These orders need to be finished by the end of the week.                  6
You receive a note indicating a personal talk with an HR specialist to discuss your work environment between 10:15AM-12:30PM.      2
Name: 1. You have just arrived to the office as an administrative worker at a large accounting firm. It is Monday, 8:30 am, and you have multiple tasks that need to get done before the day is done. Most of the staff in your office are very busy doing other things and will be unable to assist you. Your workday will be over at 2:00 PM.Which of the following tasks would you accomplish first? , dtype: int64

In [117]:
def coding(col, codeDict):
  colCoded = pd.Series(col, copy=True)
  for key, value in codeDict.items():
    colCoded.replace(key, value, inplace=True)
  return colCoded


data["1_Coded"] = coding(data["1. You have just arrived to the office as an administrative worker at a large accounting firm. It is Monday, 8:30 am, and you have multiple tasks that need to get done before the day is done. Most of the staff in your office are very busy doing other things and will be unable to assist you. Your workday will be over at 2:00 PM.Which of the following tasks would you accomplish first? "], {'You receive an Email indicating an impromptu administrative staff meeting starting in 15 minutes.':0,'There are 17 purchase orders to review for completeness. These orders need to be finished by the end of the week.':1,'You receive a note indicating a personal talk with an HR specialist to discuss your work environment between 10:15AM-12:30PM.':2})

In [118]:
data['1_Coded'].value_counts()

0    392
1      6
2      2
Name: 1_Coded, dtype: int64

In [119]:
data['2. You are a department manager and you have recently thought of a new procedure that you believe would improve the work process. Some of the employees in your department agree with the change and some do not.One of your employees openly criticizes the idea to your director. '].value_counts()

You reprimand the employee for going over your head to the director and work to promote your idea with even more enthusiasm          397
Employees' trust in their manager is important so you decide to implement only some of the changes to keep my employees satisfied      3
Name: 2. You are a department manager and you have recently thought of a new procedure that you believe would improve the work process. Some of the employees in your department agree with the change and some do not.One of your employees openly criticizes the idea to your director. , dtype: int64

In [120]:
data["2_coded"]=coding(data["2. You are a department manager and you have recently thought of a new procedure that you believe would improve the work process. Some of the employees in your department agree with the change and some do not.One of your employees openly criticizes the idea to your director. "],{'You reprimand the employee for going over your head to the director and work to promote your idea with even more enthusiasm':0,"Employees' trust in their manager is important so you decide to implement only some of the changes to keep my employees satisfie":1})

In [121]:
data['2_coded'].value_counts()

0                                                                                                                                    397
Employees' trust in their manager is important so you decide to implement only some of the changes to keep my employees satisfied      3
Name: 2_coded, dtype: int64

In [122]:
del data['2_coded']

In [123]:
data["2_coded"]=coding(data["2. You are a department manager and you have recently thought of a new procedure that you believe would improve the work process. Some of the employees in your department agree with the change and some do not.One of your employees openly criticizes the idea to your director. "],{'You reprimand the employee for going over your head to the director and work to promote your idea with even more enthusiasm':0,"Employees' trust in their manager is important so you decide to implement only some of the changes to keep my employees satisfied":1})

In [124]:
data['2_coded'].value_counts()

0    397
1      3
Name: 2_coded, dtype: int64

In [125]:
data['3. At a marketing meeting with your supervisor and the senior marketing manager you find yourself in the midst of a conflict between them. You know that the two do not get along professionally and that they are inconstant disagreement.They are now arguing about strategies for a new campaign, and are asking you to pick a side. '].value_counts()

You weigh up the advantages and disadvantages of each side and decide accordingly without getting involved in their personal conflict                                                                        392
You believe that getting involved in this dispute would be detrimental to you as both sides are superior to you. Therefore, you refuse to pick a side saying that both strategies are equally successful.      7
You accept the idea of your supervisor. Since he is directly above you, he has more influence on your daily routine in the company and therefore it is politically wiser to support him.                       1
Name: 3. At a marketing meeting with your supervisor and the senior marketing manager you find yourself in the midst of a conflict between them. You know that the two do not get along professionally and that they are inconstant disagreement.They are now arguing about strategies for a new campaign, and are asking you to pick a side. , dtype: int64

In [126]:
data['3_coded']= coding(data['3. At a marketing meeting with your supervisor and the senior marketing manager you find yourself in the midst of a conflict between them. You know that the two do not get along professionally and that they are inconstant disagreement.They are now arguing about strategies for a new campaign, and are asking you to pick a side. '],{'You weigh up the advantages and disadvantages of each side and decide accordingly without getting involved in their personal conflict':0,'You believe that getting involved in this dispute would be detrimental to you as both sides are superior to you. Therefore, you refuse to pick a side saying that both strategies are equally successful.':1,'You accept the idea of your supervisor. Since he is directly above you, he has more influence on your daily routine in the company and therefore it is politically wiser to support him.':2})

In [127]:
data['3_coded'].value_counts()

0    392
1      7
2      1
Name: 3_coded, dtype: int64

In [128]:
data['4. "You have been working as a salesperson for the past year and have consistently achieved great sales numbers. Due to personal reasons, you have recently been unfocused at work and as a result your work performance has declined. Additionally, due to changes in the market, the sales figures of your team have decreased as well.Your director does not seem to be taking the changes in the market into account and is blaming your poor performance on poor leadership on your part. "'].value_counts()

"You explain your personal situation to the director and apologise for the decline in the performance of my team. You ask to take a few days off to recuperate"    398
"You scold your team members for their poor performance and set new, more attainable sales targets in line with the changes in the market."                          2
Name: 4. "You have been working as a salesperson for the past year and have consistently achieved great sales numbers. Due to personal reasons, you have recently been unfocused at work and as a result your work performance has declined. Additionally, due to changes in the market, the sales figures of your team have decreased as well.Your director does not seem to be taking the changes in the market into account and is blaming your poor performance on poor leadership on your part. ", dtype: int64

In [129]:
data["4_coded"]=coding(data["4. \"You have been working as a salesperson for the past year and have consistently achieved great sales numbers. Due to personal reasons, you have recently been unfocused at work and as a result your work performance has declined. Additionally, due to changes in the market, the sales figures of your team have decreased as well.Your director does not seem to be taking the changes in the market into account and is blaming your poor performance on poor leadership on your part. \""],{'"You explain your personal situation to the director and apologise for the decline in the performance of my team. You ask to take a few days off to recuperate"':0,'"You scold your team members for their poor performance and set new, more attainable sales targets in line with the changes in the market."':1,})

In [130]:
data['4_coded'].value_counts()

0    398
1      2
Name: 4_coded, dtype: int64

In [131]:
data['5. "You\'ve been working in the same place for the past three years and have managed to work your way up. Lately, you have been feeling that you have reached your potential in the company so you start pursuing options for advancing your career in other companies. You are now in the midst of negotiations for a new position. Rumors that you are leaving have spread in your current work place. "'].value_counts()

"Since it is only a rumor, you don't update anyone until you actually hand in your notice. Nothing has been decided yet. "                                          386
Because the rumor is out you update your manager and only him about your intention of leaving. Since you are still an employee there, you keep working normally.     13
"Since the rumour is already out and you will probably leave, you invest a little less in your work and a bit more in attaining the new position."                    1
Name: 5. "You've been working in the same place for the past three years and have managed to work your way up. Lately, you have been feeling that you have reached your potential in the company so you start pursuing options for advancing your career in other companies. You are now in the midst of negotiations for a new position. Rumors that you are leaving have spread in your current work place. ", dtype: int64

In [132]:
a="Since it is only a rumor, you don't update anyone until you actually hand in your notice. Nothing has been decided yet. "
a

"Since it is only a rumor, you don't update anyone until you actually hand in your notice. Nothing has been decided yet. "

In [133]:

data['5_coded']= coding(data['5. "You\'ve been working in the same place for the past three years and have managed to work your way up. Lately, you have been feeling that you have reached your potential in the company so you start pursuing options for advancing your career in other companies. You are now in the midst of negotiations for a new position. Rumors that you are leaving have spread in your current work place. "'],{a:0,'Because the rumor is out you update your manager and only him about your intention of leaving. Since you are still an employee there, you keep working normally.':1,'"Since the rumour is already out and you will probably leave, you invest a little less in your work and a bit more in attaining the new position."':2})

In [134]:
data['5_coded'].value_counts()

"Since it is only a rumor, you don't update anyone until you actually hand in your notice. Nothing has been decided yet. "    386
1                                                                                                                              13
2                                                                                                                               1
Name: 5_coded, dtype: int64

In [135]:
del data['5_coded']

In [136]:
data['6. The company you work for is experiencing financial difficulties. You have thought of a creative solution that will enable it to recruit more clients.However, the downside is that the company will have to let go a stable,loyal but not so profitable client (due to a conflict of interest).Two out of three marketing people agree with you while your manager does not because she believes this is too risky. '].value_counts()

"You confront your manager on the issue and insist that she accept your proposal. You are positive that you are right. You have the marketing people to back you and will not give up until you convince your manager otherwise. "    395
"You implement your proposition despite the manager's resistance. Since you have a lot of faith in this proposition, you decide to trust your judgement and go behind her back this once for the benefit of the company."               5
Name: 6. The company you work for is experiencing financial difficulties. You have thought of a creative solution that will enable it to recruit more clients.However, the downside is that the company will have to let go a stable,loyal but not so profitable client (due to a conflict of interest).Two out of three marketing people agree with you while your manager does not because she believes this is too risky. , dtype: int64

In [137]:

data['6_coded']=coding(data['6. The company you work for is experiencing financial difficulties. You have thought of a creative solution that will enable it to recruit more clients.However, the downside is that the company will have to let go a stable,loyal but not so profitable client (due to a conflict of interest).Two out of three marketing people agree with you while your manager does not because she believes this is too risky. '],{'"You confront your manager on the issue and insist that she accept your proposal. You are positive that you are right. You have the marketing people to back you and will not give up until you convince your manager otherwise. "':0,})

In [138]:
data['6_coded'].unique()

array([0,
       '"You implement your proposition despite the manager\'s resistance. Since you have a lot of faith in this proposition, you decide to trust your judgement and go behind her back this once for the benefit of the company."'],
      dtype=object)

In [139]:
data['6_coded'].value_counts()

0                                                                                                                                                                                                                            395
"You implement your proposition despite the manager's resistance. Since you have a lot of faith in this proposition, you decide to trust your judgement and go behind her back this once for the benefit of the company."      5
Name: 6_coded, dtype: int64

In [140]:
data['7. "A co-worker is undermining you. Currently, he is at a less senior position than you in the company and hasn\'t been working there for long. However,he is better educated than you and is also considered a fast learner. You know from a third party that he is interested in taking over some of your responsibilities. "'].value_counts()

"You call your co-worker in for a talk. You explain that cooperation is an essential feature of any workplace and tell him you believe both of you can learn from one another. If he refuses to understand you take more serious steps. "    389
"Since you do not want to make a "big deal" out of the issue, you ask a third party to intimate to your co-worker that his behaviour is unacceptable. "                                                                                       11
Name: 7. "A co-worker is undermining you. Currently, he is at a less senior position than you in the company and hasn't been working there for long. However,he is better educated than you and is also considered a fast learner. You know from a third party that he is interested in taking over some of your responsibilities. ", dtype: int64

In [141]:
data['7_coded']=coding(data['7. "A co-worker is undermining you. Currently, he is at a less senior position than you in the company and hasn\'t been working there for long. However,he is better educated than you and is also considered a fast learner. You know from a third party that he is interested in taking over some of your responsibilities. "'],{'"You call your co-worker in for a talk. You explain that cooperation is an essential feature of any workplace and tell him you believe both of you can learn from one another. If he refuses to understand you take more serious steps. "':0,'"Since you do not want to make a "big deal" out of the issue, you ask a third party to intimate to your co-worker that his behaviour is unacceptable. "':1})

In [142]:
data['7_coded'].value_counts()

0    389
1     11
Name: 7_coded, dtype: int64

In [143]:
data['8. "You are a department manager. Two members of your staff are long overdue to participate in a professional training course. The training department manager has informed you that she has chosen employees from a different department instead.Relations with the training department are already problematic. "'].value_counts()

"You talk to the training manager and try to understand the reasons for her decision. You explain the necessity of training for your department's performance and try to persuade her to let at least one of your employees attend the course. "    386
"You contact the manager of the other department, whose workers were selected for training instead of your employees. You try to convince him to let one of your employees take the course instead of one of his. "                                  13
"Since relations between you and the training manager are already strained, you decide to let the subject go this time and wait a few months until the next course opens. "                                                                           1
Name: 8. "You are a department manager. Two members of your staff are long overdue to participate in a professional training course. The training department manager has informed you that she has chosen employees from a different department instead.Relation

In [144]:
data['8_coded']=coding(data['8. "You are a department manager. Two members of your staff are long overdue to participate in a professional training course. The training department manager has informed you that she has chosen employees from a different department instead.Relations with the training department are already problematic. "'],{'"You contact the manager of the other department, whose workers were selected for training instead of your employees. You try to convince him to let one of your employees take the course instead of one of his. "':1,'"Since relations between you and the training manager are already strained, you decide to let the subject go this time and wait a few months until the next course opens. "':2})

In [145]:
data['8_coded'].unique()

array(['"You talk to the training manager and try to understand the reasons for her decision. You explain the necessity of training for your department\'s performance and try to persuade her to let at least one of your employees attend the course. "',
       1, 2], dtype=object)

In [146]:
data['9. "At a marketing meeting with your supervisor and the senior marketing manager you find yourself in the midst of a conflict between them. You know that the two do not get along professionally and that they are inconstant disagreement.They are now arguing about strategies for a new campaign, and are asking you to pick a side. "'].value_counts()

"You weigh up the advantages and disadvantages of each side and decide accordingly without getting involved in their personal conflict"                                                                         391
"You believe that getting involved in this dispute would be detrimental to you as both sides are superior to you. Therefore, you refuse to pick a side saying that both strategies are equally successful. "      9
Name: 9. "At a marketing meeting with your supervisor and the senior marketing manager you find yourself in the midst of a conflict between them. You know that the two do not get along professionally and that they are inconstant disagreement.They are now arguing about strategies for a new campaign, and are asking you to pick a side. ", dtype: int64

In [147]:
data['9_coded']=coding(data['9. "At a marketing meeting with your supervisor and the senior marketing manager you find yourself in the midst of a conflict between them. You know that the two do not get along professionally and that they are inconstant disagreement.They are now arguing about strategies for a new campaign, and are asking you to pick a side. "'],{'"You weigh up the advantages and disadvantages of each side and decide accordingly without getting involved in their personal conflict"':0,'"You believe that getting involved in this dispute would be detrimental to you as both sides are superior to you. Therefore, you refuse to pick a side saying that both strategies are equally successful. "':1})

In [148]:
data['9_coded'].value_counts()

0    391
1      9
Name: 9_coded, dtype: int64

In [149]:
data['10. "After you have served two years as manager of the sales team, the director of your company appoints a new deputy manager. Although you have been able to work together, your impressions of her are negative - you find her arrogant and disloyal.The director has now considered sending her on a course that would create an opportunity for her relocation to a different position within the company. However, it would also speed up her promotion. "'].value_counts()

"You veto her participation in the course and discuss it with her. You express your concerns and you try to work out your differences. You update your director. "    394
"You approve her participation in the course since it was offered by the director. However, you voice your concerns to the director. "                                  6
Name: 10. "After you have served two years as manager of the sales team, the director of your company appoints a new deputy manager. Although you have been able to work together, your impressions of her are negative - you find her arrogant and disloyal.The director has now considered sending her on a course that would create an opportunity for her relocation to a different position within the company. However, it would also speed up her promotion. ", dtype: int64

In [150]:
data['10_coded']=coding(data['10. "After you have served two years as manager of the sales team, the director of your company appoints a new deputy manager. Although you have been able to work together, your impressions of her are negative - you find her arrogant and disloyal.The director has now considered sending her on a course that would create an opportunity for her relocation to a different position within the company. However, it would also speed up her promotion. "'],{'"You veto her participation in the course and discuss it with her. You express your concerns and you try to work out your differences. You update your director. "':0,'"You approve her participation in the course since it was offered by the director. However, you voice your concerns to the director. "':1,})

In [151]:
data['11. "You have been working as a salesperson for the past year and have consistently achieved great sales numbers. Due to personal reasons, you have recently been unfocused at work and as a result your work performance has declined. Additionally, due to changes in the market, the sales figures of your team have decreased as well.Your director does not seem to be taking the changes in the market into account and is blaming your poor performance on poor leadership on your part. "'].value_counts()

"You explain your personal situation to the director and apologise for the decline in the performance of my team. You ask to take a few days off to recuperate"    393
"You scold your team members for their poor performance and set new, more attainable sales targets in line with the changes in the market."                          7
Name: 11. "You have been working as a salesperson for the past year and have consistently achieved great sales numbers. Due to personal reasons, you have recently been unfocused at work and as a result your work performance has declined. Additionally, due to changes in the market, the sales figures of your team have decreased as well.Your director does not seem to be taking the changes in the market into account and is blaming your poor performance on poor leadership on your part. ", dtype: int64

In [152]:
data['11_coded']=coding(data['11. "You have been working as a salesperson for the past year and have consistently achieved great sales numbers. Due to personal reasons, you have recently been unfocused at work and as a result your work performance has declined. Additionally, due to changes in the market, the sales figures of your team have decreased as well.Your director does not seem to be taking the changes in the market into account and is blaming your poor performance on poor leadership on your part. "'],{'"You explain your personal situation to the director and apologise for the decline in the performance of my team. You ask to take a few days off to recuperate"':0,'"You scold your team members for their poor performance and set new, more attainable sales targets in line with the changes in the market."':1})

In [153]:
data['12. "You\'ve been working in the same place for the past three years and have managed to work your way up. Lately, you have been feeling that you have reached your potential in the company so you start pursuing options for advancing your career in other companies. You are now in the midst of negotiations for a new position. Rumors that you are leaving have spread in your current work place. "'].value_counts()

"Because the rumour is out you update your manager and only him about your intention of leaving. Since you are still an employee there, you keep working normally. "    392
"Since the rumour is already out and you will probably leave, you invest a little less in your work and a bit more in attaining the new position."                        8
Name: 12. "You've been working in the same place for the past three years and have managed to work your way up. Lately, you have been feeling that you have reached your potential in the company so you start pursuing options for advancing your career in other companies. You are now in the midst of negotiations for a new position. Rumors that you are leaving have spread in your current work place. ", dtype: int64

In [154]:
data['12_coded']=coding(data['12. "You\'ve been working in the same place for the past three years and have managed to work your way up. Lately, you have been feeling that you have reached your potential in the company so you start pursuing options for advancing your career in other companies. You are now in the midst of negotiations for a new position. Rumors that you are leaving have spread in your current work place. "'],{'"Because the rumour is out you update your manager and only him about your intention of leaving. Since you are still an employee there, you keep working normally. "':0,'"Since the rumour is already out and you will probably leave, you invest a little less in your work and a bit more in attaining the new position."':1})

In [155]:
data['13. "The company you work for is experiencing financial difficulties. You have thought of a creative solution that will enable it to recruit more clients.However, the downside is that the company will have to let go a stable,loyal but not so profitable client (due to a conflict of interest).Two out of three marketing people agree with you while your manager does not because she believes this is too risky. "'].value_counts()

"You present a document to your manager systematically detailing the advantages of your proposition and its contribution to the company. If she keeps insisting, you'll support her decision. "                                       389
"You confront your manager on the issue and insist that she accept your proposal. You are positive that you are right. You have the marketing people to back you and will not give up until you convince your manager otherwise. "     11
Name: 13. "The company you work for is experiencing financial difficulties. You have thought of a creative solution that will enable it to recruit more clients.However, the downside is that the company will have to let go a stable,loyal but not so profitable client (due to a conflict of interest).Two out of three marketing people agree with you while your manager does not because she believes this is too risky. ", dtype: int64

In [156]:
data['14. "A co-worker is undermining you. Currently, he is at a less senior position than you in the company and hasn\'t been working there for long. However,he is better educated than you and is also considered a fast learner. You know from a third party that he is interested in taking over some of your responsibilities. "'].value_counts()

"You call your co-worker in for a talk. You explain that cooperation is an essential feature of any workplace and tell him you believe both of you can learn from one another. If he refuses to understand you take more serious steps. "    388
"Since you do not want to make a "big deal" out of the issue, you ask a third party to intimate to your co-worker that his behaviour is unacceptable. "                                                                                       12
Name: 14. "A co-worker is undermining you. Currently, he is at a less senior position than you in the company and hasn't been working there for long. However,he is better educated than you and is also considered a fast learner. You know from a third party that he is interested in taking over some of your responsibilities. ", dtype: int64

In [157]:
data['14_coded']=coding(data['14. "A co-worker is undermining you. Currently, he is at a less senior position than you in the company and hasn\'t been working there for long. However,he is better educated than you and is also considered a fast learner. You know from a third party that he is interested in taking over some of your responsibilities. "'],{'"You call your co-worker in for a talk. You explain that cooperation is an essential feature of any workplace and tell him you believe both of you can learn from one another. If he refuses to understand you take more serious steps. "':0,'"Since you do not want to make a "big deal" out of the issue, you ask a third party to intimate to your co-worker that his behaviour is unacceptable. "':1})

In [158]:
data['15. "You are a department manager. Two members of your staff are long overdue to participate in a professional training course. The training department manager has informed you that she has chosen employees from a different department instead.Relations with the training department are already problematic. "'].value_counts()

"You contact the manager of the other department, whose workers were selected for training instead of your employees. You try to convince him to let one of your employees take the course instead of one of his. "                                 384
"You talk to the training manager and try to understand the reasons for her decision. You explain the necessity of training for your department's performance and try to persuade her to let at least one of your employees attend the course. "     16
Name: 15. "You are a department manager. Two members of your staff are long overdue to participate in a professional training course. The training department manager has informed you that she has chosen employees from a different department instead.Relations with the training department are already problematic. ", dtype: int64

In [159]:
data['16. "In the past month one of your employees has shown a major decline in sales performance. Although the decline has been ongoing for a couple of months, it has been particularly steep during the past two weeks. In addition, she has started coming in late and seems very frustrated with her work. Her behavior is influencing the atmosphere in the office as she is a popular employee and has been working for the company for the past two years. "'].value_counts()

"You meet with the employee and explain that her negative behaviour is affecting not only her performance but that of the entire office. You express a sincere desire to help her during her rough patch on condition that she cooperate with you and improve her attitude"    386
"You decide to let it blow over. She has been a consistently good employee until recently and you feel you need to "cut her some slack"". Part of your job is to be sensitive to your employees' needs when necessary. "                                                        14
Name: 16. "In the past month one of your employees has shown a major decline in sales performance. Although the decline has been ongoing for a couple of months, it has been particularly steep during the past two weeks. In addition, she has started coming in late and seems very frustrated with her work. Her behavior is influencing the atmosphere in the office as she is a popular employee and has been working for the company for the past two years.

In [160]:
data['17. "You are assigned to work on a project along with another co-worker. He has been working in the department longer than you. Since he has no great ambitions or aspirations to develop professionally, he does not put much effort into your mutual project. "'].value_counts()

"You reach out to the co-worker, explain your difficulty and try to negotiate a fair distribution of the work load between you. You take into account that you might have to put in extra hours to complete whatever tasks he neglects to do."    376
"You cannot accept this kind of attitude. You immediately contact your manager to discuss the situation. You request that your co-worker be replaced by another worker to complete the project. "                                                  24
Name: 17. "You are assigned to work on a project along with another co-worker. He has been working in the department longer than you. Since he has no great ambitions or aspirations to develop professionally, he does not put much effort into your mutual project. ", dtype: int64

In [161]:
data['17_coded']=coding(data['17. "You are assigned to work on a project along with another co-worker. He has been working in the department longer than you. Since he has no great ambitions or aspirations to develop professionally, he does not put much effort into your mutual project. "'],{'"You reach out to the co-worker, explain your difficulty and try to negotiate a fair distribution of the work load between you. You take into account that you might have to put in extra hours to complete whatever tasks he neglects to do."':0,'"You cannot accept this kind of attitude. You immediately contact your manager to discuss the situation. You request that your co-worker be replaced by another worker to complete the project. "':1})

In [162]:
data['18. "You work in a large audit firm as a consultant. There are five additional members in the team, all working under the supervision of a team manager and the department manager above him.Matt, a co-worker in your staff with whom you share an office, needs your advice regarding a report he is about to present at this week\'s team meeting. These meetings are led by your team manager but the department manager is almost always present, too.Although the part of the report that Matt shows you seems fine, you notice that the numerical analysis in another section of the report is missing important conclusions. You can tell that this section of the report does not adhere to the standards of your department\'s supervisors.Matt seems to be confident about that part of the report and you get the impression that he is not interested in your opinion about it. "'].value_counts()

"You do your best to get him to listen. It may involve some discomfort but you try to explain the logic behind your criticism in the hope that he will understand."    387
"It's best to stay out of it. If Matt does not want to hear your thoughts, that is his right and there is always the possibility that you are mistaken"                 13
Name: 18. "You work in a large audit firm as a consultant. There are five additional members in the team, all working under the supervision of a team manager and the department manager above him.Matt, a co-worker in your staff with whom you share an office, needs your advice regarding a report he is about to present at this week's team meeting. These meetings are led by your team manager but the department manager is almost always present, too.Although the part of the report that Matt shows you seems fine, you notice that the numerical analysis in another section of the report is missing important conclusions. You can tell that this section of the re

In [163]:
data['19. "Your organisation has implemented a new performance management system which explicitly links your performance to your pay.Your progression up the pay scale now relies on your proving you have achieved particular levels of performance."'].value_counts()

Make a note of everything you do which you think might demonstrate your performance.                                                                       385
"Spend your performance reviews complaining to your manager that the new system is unfair, and refuse to participate in any discussions relating to it.     15
Name: 19. "Your organisation has implemented a new performance management system which explicitly links your performance to your pay.Your progression up the pay scale now relies on your proving you have achieved particular levels of performance.", dtype: int64

In [164]:
data['19_coded']=coding(data['19. "Your organisation has implemented a new performance management system which explicitly links your performance to your pay.Your progression up the pay scale now relies on your proving you have achieved particular levels of performance."'],{'Make a note of everything you do which you think might demonstrate your performance.':0,'"Spend your performance reviews complaining to your manager that the new system is unfair, and refuse to participate in any discussions relating to it.':1})

In [165]:
data['20. "You answer the phone to an irate customer who claims that she has been overcharged.Although you have checked her account and cannot find any problems she remains convinced there has been a mistake and is becoming increasingly angry."'].value_counts()

Go and find your manager and let them manage the situation.                                             225
Hang up on the customer, you’ve done your best and it is inappropriate for her to behave in that way    175
Name: 20. "You answer the phone to an irate customer who claims that she has been overcharged.Although you have checked her account and cannot find any problems she remains convinced there has been a mistake and is becoming increasingly angry.", dtype: int64

In [166]:
data['20_coded']=coding(data['20. "You answer the phone to an irate customer who claims that she has been overcharged.Although you have checked her account and cannot find any problems she remains convinced there has been a mistake and is becoming increasingly angry."'],{'Go and find your manager and let them manage the situation.':0,'Hang up on the customer, you’ve done your best and it is inappropriate for her to behave in that way':1})

In [167]:
data['21. "You find a colleague crying in the toilets."  How will you react?'].value_counts()

Go and give them a big hug and tell them that everything will be ok.    378
Walk out of the toilets and leave them to it.                            22
Name: 21. "You find a colleague crying in the toilets."  How will you react?, dtype: int64

In [168]:
data['21_coded']=coding(data['21. "You find a colleague crying in the toilets."  How will you react?'],{'Go and give them a big hug and tell them that everything will be ok.':0,'Walk out of the toilets and leave them to it.':1})

In [169]:
data['22. "Everyone in your department has received a new computer system except for you." What  is your reaction ?'].value_counts()

Assume this is a mistake and speak to your manager.    369
Complain to the Human Resources department.             31
Name: 22. "Everyone in your department has received a new computer system except for you." What  is your reaction ?, dtype: int64

In [170]:
data['22_coded']=coding(data['22. "Everyone in your department has received a new computer system except for you." What  is your reaction ?'],{'Assume this is a mistake and speak to your manager.':0,'Complain to the Human Resources department.':1})

In [171]:
data['23. You are aware that large amounts of company property have been going missing over the past couple of weeks. You have noticed one of your colleagues putting stationery and other equipment from the office into her bag on a number of occasions and suspect that she is responsible.'].value_counts()

Inform your manager that you suspect your colleague is stealing.    375
Don't do anything. If guilty your colleague will be caught.          24
Confront your colleague and ask her about what you have noticed.      1
Name: 23. You are aware that large amounts of company property have been going missing over the past couple of weeks. You have noticed one of your colleagues putting stationery and other equipment from the office into her bag on a number of occasions and suspect that she is responsible., dtype: int64

In [172]:
data['23_coded']=coding(data['23. You are aware that large amounts of company property have been going missing over the past couple of weeks. You have noticed one of your colleagues putting stationery and other equipment from the office into her bag on a number of occasions and suspect that she is responsible.'],{'Inform your manager that you suspect your colleague is stealing.':0,"Don't do anything. If guilty your colleague will be caught.":1,'Confront your colleague and ask her about what you have noticed.':2})

In [173]:
data['24. You are a Project Leader and manage the coordination of four Team Members. Two Team Members won’t be able to meet the deadlines of the project. The Director of the organisation reminds you the importance of meeting deadlines. Which is the most likely course of action you would take?'].value_counts()

Discuss with the two Team Members that are having difficulties, identify the causes, evaluate the remaining workload and determine an action plan to finish the project.         373
Inform your Director of the current situation of the Team Members that are having difficulties, so as to justify the status on the project, and ask for additional resources.     26
Ask the two Team Members that are late, to work additional hours in order to respect the deadlines.                                                                                1
Name: 24. You are a Project Leader and manage the coordination of four Team Members. Two Team Members won’t be able to meet the deadlines of the project. The Director of the organisation reminds you the importance of meeting deadlines. Which is the most likely course of action you would take?, dtype: int64

In [174]:
data['24_coded']=coding(data['24. You are a Project Leader and manage the coordination of four Team Members. Two Team Members won’t be able to meet the deadlines of the project. The Director of the organisation reminds you the importance of meeting deadlines. Which is the most likely course of action you would take?'],{'Discuss with the two Team Members that are having difficulties, identify the causes, evaluate the remaining workload and determine an action plan to finish the project.':0,'Inform your Director of the current situation of the Team Members that are having difficulties, so as to justify the status on the project, and ask for additional resources.':1,'Ask the two Team Members that are late, to work additional hours in order to respect the deadlines.':2})

In [175]:
data['25. You have been assigned to work on a global project for your employer. You have to work with a member of another Unit. The Head of Unit has determined a fixed plan of action for each step. After a couple of weeks, you notice that your colleague on the project is not following the plan and his contribution is low, which is starting to have a negative effect on the project. Which is the most likely course of action you would take?'].value_counts()

Discuss it with your colleague, ask him the reasons for the delay and fix up together a plan to continue on the project.    391
Send an email with a status report on the project, indicate what you did, with your Head of Unit in cc.                       9
Name: 25. You have been assigned to work on a global project for your employer. You have to work with a member of another Unit. The Head of Unit has determined a fixed plan of action for each step. After a couple of weeks, you notice that your colleague on the project is not following the plan and his contribution is low, which is starting to have a negative effect on the project. Which is the most likely course of action you would take?, dtype: int64

In [176]:
data['25_coded']=coding(data['25. You have been assigned to work on a global project for your employer. You have to work with a member of another Unit. The Head of Unit has determined a fixed plan of action for each step. After a couple of weeks, you notice that your colleague on the project is not following the plan and his contribution is low, which is starting to have a negative effect on the project. Which is the most likely course of action you would take?'],{'Discuss it with your colleague, ask him the reasons for the delay and fix up together a plan to continue on the project.':0,'Send an email with a status report on the project, indicate what you did, with your Head of Unit in cc.':1})

In [177]:
predictors=['1_Coded','3_coded','4_coded','7_coded','9_coded','10_coded','11_coded','12_coded','14_coded','17_coded','19_coded','20_coded','21_coded','22_coded','23_coded','24_coded','25_coded']

In [178]:
X=data[predictors]
y=data['Select a Role']

In [179]:
import sklearn


In [180]:
!pip install sklearn.cross_validation


  Could not find a version that satisfies the requirement sklearn.cross_validation (from versions: )
No matching distribution found for sklearn.cross_validation
You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [183]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

In [184]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

In [185]:
logmodel = LogisticRegression()

In [186]:
logmodel.fit(X_train,y_train)

c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [187]:
predictions = logmodel.predict(X_test)

In [188]:
from sklearn.metrics import classification_report

In [189]:
print(classification_report(y_test,predictions))

                   precision    recall  f1-score   support

          Analyst       0.00      0.00      0.00        12
       Consultant       0.33      0.33      0.33        15
         Designer       0.00      0.00      0.00        16
          Manager       0.17      0.05      0.08        19
            Sales       0.18      0.54      0.27        13
Software Engineer       0.00      0.00      0.00        19
Technical Support       0.12      0.11      0.12         9
           Tester       0.29      0.65      0.40        17

        micro avg       0.21      0.21      0.21       120
        macro avg       0.14      0.21      0.15       120
     weighted avg       0.14      0.21      0.15       120



In [190]:
from sklearn.metrics import confusion_matrix

In [191]:
confusion_matrix(y_test,predictions)

array([[ 0,  1,  1,  1,  2,  0,  1,  6],
       [ 1,  5,  0,  0,  6,  2,  0,  1],
       [ 2,  2,  0,  1,  7,  1,  0,  3],
       [ 1,  1,  0,  1, 10,  1,  1,  4],
       [ 0,  2,  0,  0,  7,  0,  1,  3],
       [ 2,  4,  0,  1,  4,  0,  0,  8],
       [ 1,  0,  1,  2,  2,  0,  1,  2],
       [ 1,  0,  1,  0,  0,  0,  4, 11]], dtype=int64)

In [192]:

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logmodel.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.21


In [212]:
##Testing Purpose

tester=[[0,1,2,3,2,1,2,3,1,2,3,1,0,2,1,0,1]]
df=pd.DataFrame(tester)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0,1,2,3,2,1,2,3,1,2,3,1,0,2,1,0,1


In [214]:
test=logmodel.predict(df)
test

array(['Designer'], dtype=object)

In [70]:
rec=pd.read_csv("recruiter_info.csv",sep=';')
std=pd.read_csv("student_info.csv",sep=';')

In [71]:
rec.head(5)

,company_location,recruiter_company,recruiter_email,required_skills,required_age,required_experience,required_role,required_educational_qualification,required_gender,recruiter_password,required_number_of_candidates
0,Banglore,Google,google@gmail.com,ML,30,3,Engineer,B.Tech,0,steve123,5
1,Mumbai,Microsoft,msn@hotmail.com,"c++,sales",32,5,Marketing,MBA,1,hotmail12,10
2,Banglore,TCS,campuscommune@gmail.com,c,21,0,Engineer,M.Tech,0,tcs124,15
3,Nagpur,LTI,larsenturbo@gmail.com,"python,Photoshop",30,2,Designer,B.A,1,art09,3
4,Chennai,Axis Bank,axis@hotmail.com,"hadoop,java,c",30,6,Technical Support,B.Tech,1,axis908,17


In [72]:
std.head(10)

,stud_id,stud_name,stud_email,stud_address,student_age,experience,skills,previous_position,education_qualification,gender,sector,area_of _interest,password
0,6,Sheetal Bhosale,sheetal.bhosale@gmail.com,Tamil Nadu,30,6,Sales,Manager,B.Tech,1,Marketing,sales,sheetal@123
1,9,Pranshu Diwan,psd@gmail.com,Sikkim,26,7,ML,Engineer,M.S,0,Education,Sales,psd@890
2,0,Habib Shaikh,Habib.shaikh@gmail.com,Mumbai,27,5,html,Manager,M.Tech,0,Bank,Gaming,habib123
3,2,Arghya Chell,fishrocks@gmail.com,Kolkata,36,15,unity,software engineer,P.H.D,0,Food,Food,anisha12
4,5,Sharvari Sonak,sharvaridcr@gmail.com,Gujarat,25,3,Management,Marketing Lead,B.Tech,1,Finance,Education,sharu@123
5,8,Rishabh Mehra,rishabhmehra5@gmail.com,Delhi,32,9,Python,Engineer,M.Tech,1,Marketing,Sales,rkm@123
6,3,Mrunal Ghadge,mg@gmail.com,Chennai,28,7,Excel,Analyst,M.S,1,Marketing,Marketing,mrunal@123
7,4,Sagar Mhatre,sagar12@gmail.com,Banglore,29,10,Photoshop,Designer,B.A,0,Education,graphics,sagar12


In [73]:
stud=std.loc[std['stud_id']==9].reindex()

In [74]:
stud

,stud_id,stud_name,stud_email,stud_address,student_age,experience,skills,previous_position,education_qualification,gender,sector,area_of _interest,password
1,9,Pranshu Diwan,psd@gmail.com,Sikkim,26,7,ML,Engineer,M.S,0,Education,Sales,psd@890


In [75]:
comp=rec.loc[rec['recruiter_company']=='Google']

In [76]:
comp

,company_location,recruiter_company,recruiter_email,required_skills,required_age,required_experience,required_role,required_educational_qualification,required_gender,recruiter_password,required_number_of_candidates
0,Banglore,Google,google@gmail.com,ML,30,3,Engineer,B.Tech,0,steve123,5


In [77]:
#abs(int(comp.required_experience)-int(stud.experience))/5
##if str(stud.skills) == str(comp.required_skills):
##    print("Hello Friends")
##else:
##    print("Chai Peelo")
type(stud.skills.values[0])

str

In [100]:
lemmatizer=WordNetLemmatizer()
ps=PorterStemmer()
b=(lemmatizer.lemmatize('Salesman'))
a=(ps.stem('salesman'))
a=a+'.n.01'
b=b+'.n.01'
w1=wordnet.synset(a)
w2=wordnet.synset(b)
print(w1.wup_similarity(w2))

1.0


In [106]:
education={
        "B.Tech":0,
        "M.Tech":1,
        "MBA":2,
        "M.S":3,
        "P.H.D":4
    }
education[comp.required_educational_qualification[0]]



0

In [101]:
def gen_score(stud_id,company_name):
    student=std.loc[std['stud_id']==stud_id]
    company= rec.loc[rec['recruiter_company']==company_name]
    aoi_st=student.area_of_intrest
    role_cmp=company.required_role
    aoi_std=ps.stem(aoi_st)+'.n.01'
    role_cmp=ps.stem(role_cmp)+'.n.01'
    aoi_st=wordnet.synset(aoi_st)
    role_cmp=wordnet.synset(role_cmp)
    if int(student.experience)>int(company.required_experience):
        exp_score=abs(int(student.experience)-int(company.required_experience))/5
    skills=list()
    skills=company.required_skills.values[0].lower().split(',')
    if student.skills.values[0] == company.required_skills.values[0]:
        skill_score=1/skills.length()
    aoi_score=aoi_st.wup_similarity(role_cmp)
    
    education={
        "B.Tech":1,
        "M.Tech":2,
        "MBA":3,
        "M.S":4
        "P.H.D":5
    }
    
    if education[student.education_qualification[0]]<education[company.required_educational_qualification]:
        edu_score=0
    else:
        edu_score=0.5+education[student.education_qualification[0]]/10
        
    